<a href="https://colab.research.google.com/github/JozefSL/2022/blob/main/colab/ND_WellDataMonthly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
import xlrd
from xlrd import XLRDError
import csv
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
from google.colab import drive
import os
import sqlite3

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:


#drive.mount('drive')
def PullNDData():
    #%%
    # create list of all yyyy-mm for data download
    ymList = pd.date_range('2015-04-10',datetime.today()- relativedelta(months=2), freq='MS').strftime("%Y_%m").tolist()  # with month in numeric format
    #ymList = pd.date_range(datetime.today()- relativedelta(months=6),datetime.today() - relativedelta(months=2), freq='MS').strftime("%Y_%m").tolist()  # with month in numeric format

    # Access the ND DMR data @ https://www.dmr.nd.gov/oilgas/mprindex.asp
    website = "https://www.dmr.nd.gov/oilgas/mpr/"

    #%%

    NDMWD = []  #list()  # North Dakota Monthly Well Data
    for month in ymList:
        try:
            target_url = website + month + '.xlsx'
            print(target_url)
            r = requests.get(target_url)
            wb = pd.read_excel(xlrd.open_workbook(file_contents=r.content))
            #print(r)
            #print(wb.head())
            NDMWD.append(wb)
        except: #finds exceptions when pulling data from website, logs and displays errors to screen. Will add to file.
            if(RuntimeError):
                print(RuntimeError)
            elif(XLRDError):
                print(XLRDError)
            print("error month: ", month)
            pass

    #%%
    #print(NDMWD)
    NDMWD = pd.concat(NDMWD)
    NDMWD.rename(columns={"API_WELLNO": "APINumber"},inplace=True)
    NDMWD['APINumber'] = NDMWD['APINumber'].astype(str) #sets APINumber column as a string/character
    #%%
    #Create ND monthly well data  
    #NDMWD.to_csv('NDMWD.csv')
    #!cp .csv "drive/My Drive/Colab Notebooks"
    #NDMWD.to_csv('drive/My Drive/Colab Notebooks/NDMWD.csv')
    
    print('ND well data query executed on', datetime.now().strftime("%A, %B %d, %Y at %I:%M%p"),'(GMT')
    return NDMWD
NDMWD = PullNDData()

https://www.dmr.nd.gov/oilgas/mpr/2015_05.xlsx
https://www.dmr.nd.gov/oilgas/mpr/2015_06.xlsx
https://www.dmr.nd.gov/oilgas/mpr/2015_07.xlsx
https://www.dmr.nd.gov/oilgas/mpr/2015_08.xlsx
https://www.dmr.nd.gov/oilgas/mpr/2015_09.xlsx
https://www.dmr.nd.gov/oilgas/mpr/2015_10.xlsx
https://www.dmr.nd.gov/oilgas/mpr/2015_11.xlsx
https://www.dmr.nd.gov/oilgas/mpr/2015_12.xlsx
https://www.dmr.nd.gov/oilgas/mpr/2016_01.xlsx
https://www.dmr.nd.gov/oilgas/mpr/2016_02.xlsx
https://www.dmr.nd.gov/oilgas/mpr/2016_03.xlsx
https://www.dmr.nd.gov/oilgas/mpr/2016_04.xlsx
https://www.dmr.nd.gov/oilgas/mpr/2016_05.xlsx
https://www.dmr.nd.gov/oilgas/mpr/2016_06.xlsx
https://www.dmr.nd.gov/oilgas/mpr/2016_07.xlsx
https://www.dmr.nd.gov/oilgas/mpr/2016_08.xlsx
https://www.dmr.nd.gov/oilgas/mpr/2016_09.xlsx
https://www.dmr.nd.gov/oilgas/mpr/2016_10.xlsx
https://www.dmr.nd.gov/oilgas/mpr/2016_11.xlsx
https://www.dmr.nd.gov/oilgas/mpr/2016_12.xlsx
https://www.dmr.nd.gov/oilgas/mpr/2017_01.xlsx
https://www.d

In [ ]:
NDMWD.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1314442 entries, 0 to 19253
Data columns (total 21 columns):
 #   Column      Non-Null Count    Dtype         
---  ------      --------------    -----         
 0   ReportDate  1314442 non-null  datetime64[ns]
 1   APINumber   1314442 non-null  object        
 2   FileNo      1314442 non-null  int64         
 3   Company     1314442 non-null  object        
 4   WellName    1314442 non-null  object        
 5   Quarter     1314326 non-null  object        
 6   Section     1314415 non-null  float64       
 7   Township    1314415 non-null  float64       
 8   Range       1314415 non-null  float64       
 9   County      1314442 non-null  object        
 10  FieldName   1314442 non-null  object        
 11  Pool        1314442 non-null  object        
 12  Oil         1304698 non-null  float64       
 13  Wtr         1304698 non-null  float64       
 14  Days        1304698 non-null  float64       
 15  Runs        1314442 non-null  int6

In [ ]:
firstProdMonth = NDMWD[['ReportDate','APINumber','Oil','Gas','Days']]
firstProdMonth = firstProdMonth.sort_values(['APINumber','ReportDate']).drop_duplicates('APINumber', keep='first')
firstProdMonth.rename({'ReportDate': 'firstProdMonth'}, axis=1, inplace=True)

In [ ]:
firstProdMonth['oP0'] = firstProdMonth['Oil']/firstProdMonth['Days']
firstProdMonth['gP0'] = firstProdMonth['Gas']/firstProdMonth['Days']
firstProdMonth = firstProdMonth.drop(['Oil','Gas','Days'],axis=1)
firstProdMonth = firstProdMonth.dropna().reset_index(drop=True)

In [ ]:
#firstProdMonth.tail()
firstProdMonth.info()
#firstProdMonth[firstProdMonth['APINumber']== '33007000400000']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15936 entries, 0 to 15935
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   firstProdMonth  15936 non-null  datetime64[ns]
 1   APINumber       15936 non-null  object        
 2   oP0             15936 non-null  float64       
 3   gP0             15936 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 498.1+ KB


In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
conn = sqlite3.connect('/content/drive/MyDrive/FF_data')
print("connected to sqlite3 verion:", sqlite3.version)
firstProdMonth.to_sql('firstProdMonth', conn, index=False, if_exists='replace')
conn.close()
print('Updated firstProdMonth table on', datetime.now().strftime("%A, %B %d, %Y at %I:%M%p"),'(GMT)' '\nconnection closed')

connected to sqlite3 verion: 2.6.0
Updated firstProdMonth table on Tuesday, December 21, 2021 at 02:15AM (GMT)
connection closed


In [4]:
conn = sqlite3.connect('/content/drive/MyDrive/FracFocus/FF_data')
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

print(cursor.fetchall())
conn.close()

[('RegistryUpload',), ('FracFocusRegistry',), ('firstProdMonth',)]


In [ ]:
os.listdir()

['.config', 'drive', 'sample_data']

In [ ]:
!ls

drive  sample_data
